In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
import numpy as np
import cv2
import time
from tensorflow.keras.applications import vgg19

# --- KONFIGURASI ---
# Nama file gambar yang diunggah
CONTENT_IMAGE_PATH = '/content/drive/MyDrive/Deep learning/Deep learning lanjut/Data gambar/ayam jago.jpg' # Gambar Ayam Jago (Content)
STYLE_IMAGE_PATH = '/content/drive/MyDrive/Deep learning/Deep learning lanjut/Data gambar/Langit kartun.jpg'  # Gambar Bintang Laut (Style)
OUTPUT_IMAGE_PATH = '/content/drive/MyDrive/Deep learning/Deep learning lanjut/Data gambar/generated_style_transfer_output.png'

# Ukuran gambar (semua gambar akan diubah ukurannya ke dimensi ini)
IMG_HEIGHT = 512
IMG_WIDTH = 512

# Layer VGG19 yang digunakan untuk menghitung Content Loss dan Style Loss
# VGG19 pretrained pada ImageNet adalah pilihan umum untuk Style Transfer
CONTENT_LAYERS = ['block5_conv2']
STYLE_LAYERS = [
    'block1_conv1',
    'block2_conv1',
    'block3_conv1',
    'block4_conv1',
    'block5_conv1'
]

# Faktor bobot untuk Content Loss, Style Loss, dan Total Variation Loss
# Mengontrol seberapa kuat gambar konten dan gaya harus dipertahankan, serta seberapa halus hasilnya.
CONTENT_WEIGHT = 1e3
STYLE_WEIGHT = 1e-2
TOTAL_VARIATION_WEIGHT = 30

# Iterasi optimasi (semakin banyak, semakin baik hasilnya, tetapi semakin lama)
NUM_ITERATIONS = 500

# --- FUNGSI UTAMA ---

def load_and_process_img(path):
    """
    Memuat gambar, mengubah ukurannya, dan mengubahnya menjadi tensor.
    """
    img = cv2.imread(path)
    if img is None:
        raise FileNotFoundError(f"File gambar tidak ditemukan: {path}")

    # Ubah dari BGR (OpenCV) ke RGB (TensorFlow/VGG)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    img = tf.convert_to_tensor(img, dtype=tf.float32)

    # Tambahkan dimensi batch (1, H, W, C)
    img = tf.expand_dims(img, axis=0)

    # Preprocessing standar VGG19: normalisasi, dll.
    img = vgg19.preprocess_input(img)
    return img

def deprocess_img(processed_img):
    """
    Mengubah tensor gambar hasil kembali ke format gambar yang dapat disimpan.
    """
    # Menghilangkan dimensi batch
    x = processed_img.numpy()[0]

    # Menghilangkan nol mean (kebalikan dari preprocessing)
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68

    # Konversi dari BGR ke RGB (sejak VGG menggunakan BGR, tetapi kita ingin output RGB)
    x = x[:, :, ::-1] # Konversi dari BGR ke RGB (karena VGG_preprocess_input membalik)

    # Batasi nilai piksel antara 0 dan 255
    x = np.clip(x, 0, 255).astype('uint8')

    # Ubah kembali ke BGR untuk penyimpanan dengan OpenCV
    return cv2.cvtColor(x, cv2.COLOR_RGB2BGR)


# --- MODEL VGG19 DAN FITUR LOSS ---

def get_model():
    """
    Membuat model Keras yang akan mengembalikan fitur dari layer Style dan Content.
    """
    # Muat model VGG19 tanpa layer klasifikasi (include_top=False)
    vgg = vgg19.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False

    # Petakan nama layer ke output tensor mereka
    style_outputs = [vgg.get_layer(name).output for name in STYLE_LAYERS]
    content_outputs = [vgg.get_layer(name).output for name in CONTENT_LAYERS]
    model_outputs = content_outputs + style_outputs

    # Bangun model yang mengambil input gambar dan mengembalikan output fitur
    return tf.keras.Model(vgg.input, model_outputs)

def gram_matrix(input_tensor):
    """
    Menghitung Gram Matrix untuk Style Loss.
    """
    # Hitung produk luar dari fitur. E.g., jika tensor 4D adalah (1, H, W, C)
    # Kita ingin (H*W, C) * (C, H*W) -> (C, C). C adalah jumlah filter (kedalaman).
    result = tf.linalg.einsum('ijkl,ijkm->ilm', input_tensor, input_tensor)

    # Hitung normalisasi (jumlah total elemen)
    input_shape = tf.shape(input_tensor)
    num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
    return result / num_locations

def get_feature_representations(model, content_path, style_path):
    """
    Mendapatkan representasi fitur (Content dan Style) dari gambar input.
    """
    # Muat dan proses gambar
    content_image = load_and_process_img(content_path)
    style_image = load_and_process_img(style_path)

    # Dapatkan output fitur dari model
    style_features = model(style_image)
    content_features = model(content_image)

    # Bagi fitur-fitur menjadi Content dan Style
    # Jumlah output konten adalah len(CONTENT_LAYERS)
    content_outputs = content_features[:len(CONTENT_LAYERS)]
    style_outputs = style_features[len(CONTENT_LAYERS):]

    # Hitung Gram Matrix untuk fitur Style
    style_grams = [gram_matrix(style_output) for style_output in style_outputs]

    # Kembalikan pasangan (fitur konten, Gram Matrix gaya)
    return (
        content_outputs,
        style_grams
    )

# --- FUNGSI LOSS ---

def content_loss(base_content, target_content):
    """
    Menghitung Content Loss (Mean Squared Error antara fitur).
    """
    return tf.reduce_mean(tf.square(base_content - target_content))

def style_loss(base_style_gram, target_style_gram):
    """
    Menghitung Style Loss (Mean Squared Error antara Gram Matrices).
    """
    # Gram Matrix sudah dinormalisasi di fungsi gram_matrix
    return tf.reduce_mean(tf.square(base_style_gram - target_style_gram))

def total_loss(model, loss_weights, init_image, content_features, style_grams):
    """
    Menghitung total loss: Content Loss + Style Loss + Total Variation Loss.
    """
    style_weight, content_weight, total_variation_weight = loss_weights

    # Preprocessing gambar yang sedang dioptimalkan
    preprocessed_image = vgg19.preprocess_input(init_image)

    # Dapatkan fitur dari gambar yang sedang dioptimalkan
    model_outputs = model(preprocessed_image)

    content_outputs = model_outputs[:len(CONTENT_LAYERS)]
    style_outputs = model_outputs[len(CONTENT_LAYERS):]

    style_score = 0
    content_score = 0

    # 1. Content Loss
    weight_per_content_layer = 1.0 / len(CONTENT_LAYERS)
    for target_content, current_content in zip(content_features, content_outputs):
        content_score += weight_per_content_layer * content_loss(target_content, current_content)

    # 2. Style Loss
    weight_per_style_layer = 1.0 / len(STYLE_LAYERS)
    for target_style_gram, current_style_output in zip(style_grams, style_outputs):
        current_style_gram = gram_matrix(current_style_output)
        style_score += weight_per_style_layer * style_loss(target_style_gram, current_style_gram)

    # 3. Total Variation Loss (untuk smoothing)
    # L2 loss pada perbedaan piksel horizontal dan vertikal
    tv_loss = tf.image.total_variation(init_image)

    # Kombinasikan loss
    loss = (content_weight * content_score) + (style_weight * style_score) + (total_variation_weight * tv_loss)
    return loss, style_score, content_score, tv_loss

# --- OPTIMASI ---

@tf.function()
def compute_grads(cfg):
    """
    Menghitung gradient dari loss terhadap gambar yang sedang dioptimalkan.
    """
    with tf.GradientTape() as tape:
        all_loss, _, _, _ = total_loss(**cfg)

    # Gradient dihitung pada input gambar
    total_grad = tape.gradient(all_loss, cfg['init_image'])
    return total_grad, all_loss

# --- FUNGSI EKSEKUSI ---

def run_style_transfer(content_path, style_path, num_iterations=NUM_ITERATIONS):
    """
    Menjalankan proses Style Transfer lengkap.
    """
    # Inisialisasi model dan dapatkan representasi fitur
    model = get_model()
    for layer in model.layers:
        layer.trainable = False

    content_features, style_grams = get_feature_representations(model, content_path, style_path)

    # Gambar awal (inisialisasi dari gambar konten)
    init_image = load_and_process_img(content_path)

    # Kita menggunakan tf.Variable agar tensor dapat dilacak oleh GradientTape
    image_to_optimize = tf.Variable(init_image)

    # Inisialisasi optimizer Adam (atau L-BFGS-B, namun Adam lebih mudah diimplementasikan di TF)
    optimizer = tf.optimizers.Adam(learning_rate=5.0, beta_1=0.99, epsilon=1e-1)

    # Konfigurasi untuk fungsi loss dan gradient
    cfg = {
      'model': model,
      'loss_weights': (STYLE_WEIGHT, CONTENT_WEIGHT, TOTAL_VARIATION_WEIGHT),
      'init_image': image_to_optimize,
      'content_features': content_features,
      'style_grams': style_grams
    }

    # Loop optimasi
    print(f"Memulai optimasi untuk {num_iterations} iterasi...")
    start_time = time.time()

    best_loss = float('inf')
    best_img = None

    for i in range(num_iterations):
        # Hitung gradient dan total loss
        grads, loss = compute_grads(cfg)

        # Aplikasikan gradient
        optimizer.apply_gradients([(grads, image_to_optimize)])

        # Batasi nilai piksel agar tetap di rentang valid VGG (0-255 RGB)
        # Dengan ini, kita memastikan gambar tetap "wajar" dan mencegah noise.
        # Catatan: VGG19 bekerja pada range [-123.68, 255-123.68] untuk R, dst.
        # Clipping pada tensor VGG pre-processed.
        min_vals = -vgg19.preprocess_input(np.zeros((1, IMG_HEIGHT, IMG_WIDTH, 3))).flatten()
        max_vals = 255 - vgg19.preprocess_input(np.zeros((1, IMG_HEIGHT, IMG_WIDTH, 3))).flatten()
        image_to_optimize.assign(tf.clip_by_value(image_to_optimize, min_vals[0], max_vals[0]))

        if loss < best_loss:
            best_loss = loss
            best_img = deprocess_img(image_to_optimize)

        if i % 100 == 0:
            print(f"Iterasi {i}: Total Loss = {loss.numpy().item():.2f}") # Fix: use .item() to get scalar value

    end_time = time.time()
    print(f"\nOptimasi selesai dalam {end_time - start_time:.2f} detik.")

    return best_img

# --- EKSEKUSI SCRIPT ---
if __name__ == '__main__':
    try:
        # Jalankan Style Transfer
        stylized_image_bgr = run_style_transfer(CONTENT_IMAGE_PATH, STYLE_IMAGE_PATH, NUM_ITERATIONS)

        # Simpan gambar hasil
        cv2.imwrite(OUTPUT_IMAGE_PATH, stylized_image_bgr)
        print(f"\n[HASIL TERSIMPAN] Gambar hasil Style Transfer telah disimpan sebagai '{OUTPUT_IMAGE_PATH}'")

        # Tampilkan beberapa info
        print(f"Gambar Konten: '{CONTENT_IMAGE_PATH}' ")
        print(f"Gambar Gaya: '{STYLE_IMAGE_PATH}' ")

    except FileNotFoundError as e:
        print(f"ERROR: {e}")
        print("Pastikan Anda sudah mengunggah file gambar dengan nama yang benar.")
    except Exception as e:
        print(f"ERROR: Terjadi kesalahan saat menjalankan Style Transfer: {e}")


80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Memulai optimasi untuk 500 iterasi...
Iterasi 0: Total Loss = 951435200.00
Iterasi 100: Total Loss = 195429600.00
Iterasi 200: Total Loss = 132622112.00
Iterasi 300: Total Loss = 100485016.00
Iterasi 400: Total Loss = 83116088.00

Optimasi selesai dalam 3458.63 detik.

[HASIL TERSIMPAN] Gambar hasil Style Transfer telah disimpan sebagai '/content/drive/MyDrive/Deep learning/Deep learning lanjut/Data gambar/generated_style_transfer_output.png'
Gambar Konten: '/content/drive/MyDrive/Deep learning/Deep learning lanjut/Data gambar/ayam jago.jpg' 
Gambar Gaya: '/content/drive/MyDrive/Deep learning/Deep learning lanjut/Data gambar/Langit kartun.jpg' 
